In [8]:
import numpy as np
from sklearn.linear_model import LinearRegression
import pandas as pd
import string
import math

# Dataset 1

In [3]:
# Dataset 1
# Found at: https://www.kaggle.com/datasets/sahasourav17/students-anxiety-and-depression-dataset/data
filepath1 = "C:/Users/28chr/Downloads/dataset.xlsx"
data1 = pd.read_excel(filepath1)

In [5]:
# Get word list from dataset 1 training set.
# word_list[word] is a tuple, with [0] being the number of uses
# in control posts and [1] being the number of uses in anxious posts.
word_list = {}
for i in range(len(data1["text"])):
    # i % 5 == 4 is the test set
    if i % 5 != 4:
        post = str(data1["text"][i])
        anxious = data1["label"][i] == 1
        # Post with all punctuation and other symbols removed
        no_punct = ""
        
        for letter in post:
            if letter in string.ascii_lowercase or letter in string.ascii_uppercase:
                no_punct += (letter)
            else:
                no_punct += (" ")
                
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                if anxious:
                    word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                else:
                    word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                if anxious:
                    word_list[word] = (0, 1)
                else:
                    word_list[word] = (1, 0)

In [6]:
# Calculate anxiety score for each word,
# that is, the odds that a usage of that word
# was by someone with anxiety in the training set
anxiety_scores = {}
for word in word_list:
    num_control = word_list[word][0]
    num_anxious = word_list[word][1]
    num_total = num_control + num_anxious
    # Threshold to limit rarely used words; May vary but should be at least 1
    thres = 1
    if num_total > thres:
        score = num_anxious / num_total
        anxiety_scores[word] = score

In [7]:
# Build arrays for linear regression model.
# Training scores consists of five values:
# Average word score, number of capital letters,
# number of punctuation symbols, post length,
# and number of rarely used words
training_scores = []
training_labels = []
for i in range(len(data1["text"])):
    if i % 5 != 4:
        caps = 0
        punct = 0
        one_offs = 0
        training_labels.append(data1["label"][i])
        post = str(data1["text"][i])
        
        # Remove punctuation from the post and count symbols and capital letters
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        
        # Calculate anciety score and number of rarely used words
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                # Keep track of the number of words that contribute to average anxiety score
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs])

In [9]:
# Clean data to avoid linear regression errors
for i in range(len(training_scores)):
    if math.isnan(training_scores[i][0]):
        training_scores[i][0] = 0
    if math.isnan(training_labels[i]):
        training_labels[i] = 0

In [10]:
# Format arrays and make a linear regression model.
# This model will output a similarity score,
# where a higher score indicates higher likelihood of anxiety.
training_scores = np.array(training_scores)
training_labels = np.array(training_labels)
reg = LinearRegression().fit(training_scores, training_labels)

In [11]:
# Test the model and record results
res = []
# Same algorithm used to get training data
for i in range(len(data1["text"])):
    # test set
    if i % 5 == 4:
        caps = 0
        punct = 0
        one_offs = 0
        label = data1["label"][i]
        post = str(data1["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        # get linear regression's prediction and add it to the results list
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs]]))
        res.append((out, label))

In [12]:
# Find ideal cutoff value based on test data
# This value can then be used to evaluate future queries
ind = 0
mini = 10000000000000000000 # arbitrary big number
for t in range (101):
    thres = t / 100
    mis = 0
    for tup in res:
        # false positive
        if tup[0][0] > thres and tup[1] == 0:
            mis += 1
        # false negative
        elif tup[0][0] < thres and tup[1] == 1:
            mis += 1
    if mis < mini:
        ind = t
        mini = mis
print("Ideal threshold:", ind / 100)
print("Number misclassified:", mini)
print("Misclassification rate:", mini / len(res))

Ideal threshold: 0.3
Number misclassified: 57
Misclassification rate: 0.04083094555873926


In [13]:
# Combined test of first dataset, using five different test sets.
# More accurately determines ideal cutoff value
# Code is the same as above, but with j for the test set number
all_res = []
for j in range(5):
    num_anxious = 0
    num_control = 0
    word_list = {}
    for i in range(len(data1["text"])):
        # i % 5 == j is the test set
        if i % 5 != j:
            post = str(data1["text"][i])
            anxious = data1["label"][i] == 1
            num_lower = 0
            num_upper = 0
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    num_lower += 1
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    num_upper += 1
                    no_punct += (letter)
                else:
                    no_punct += (" ")
            for word in no_punct.split(" "):
                word = word.lower()
                if word in word_list:
                    if anxious:
                        word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                    else:
                        word_list[word] = (word_list[word][0] + 1, word_list[word][1])
                else:
                    if anxious:
                        word_list[word] = (0, 1)
                    else:
                        word_list[word] = (1, 0)
            num_punct = len(post) - num_upper - num_lower
            if anxious:
                num_anxious += 1
            else:
                num_control += 1
    anxiety_scores = {}
    for word in word_list:
        num_control = word_list[word][0]
        num_anxious = word_list[word][1]
        num_total = num_control + num_anxious
        if num_total > 1:
            score = num_anxious / num_total
            anxiety_scores[word] = score
    training_scores = []
    training_labels = []
    for i in range(len(data1["text"])):
        if i % 5 != j:
            caps = 0
            punct = 0
            one_offs = 0
            training_labels.append(data1["label"][i])
            post = str(data1["text"][i])
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    no_punct += letter
                    caps += 1
                else:
                    no_punct += " "
                    punct += 1
            score = 0.0
            words = no_punct.split(" ")
            avg_divisor = len(words)
            for word in words:
                word = word.lower()
                if word in anxiety_scores:
                    score += anxiety_scores[word]
                else:
                    avg_divisor -= 1
                    one_offs += 1
            if avg_divisor == 0:
                score = 0
                avg_divisor = 1
            avg_score = score / avg_divisor
            training_scores.append([avg_score, caps, punct, len(post), one_offs])
    for i in range(len(training_scores)):
        if math.isnan(training_scores[i][0]):
            training_scores[i][0] = 0
        if math.isnan(training_labels[i]):
            training_labels[i] = 0
    training_scores = np.array(training_scores)
    training_labels = np.array(training_labels)
    reg = LinearRegression().fit(training_scores, training_labels)
    res = []
    for i in range(len(data1["text"])):
        if i % 5 == j:
            caps = 0
            punct = 0
            one_offs = 0
            label = data1["label"][i]
            post = str(data1["text"][i])
            no_punct = ""
            for letter in post:
                if letter in string.ascii_lowercase:
                    no_punct += (letter)
                elif letter in string.ascii_uppercase:
                    no_punct += letter
                    caps += 1
                else:
                    no_punct += " "
                    punct += 1
            score = 0.0
            words = no_punct.split(" ")
            avg_divisor = len(words)
            for word in words:
                word = word.lower()
                if word in anxiety_scores:
                    score += anxiety_scores[word]
                else:
                    avg_divisor -= 1
                    one_offs += 1
            if avg_divisor == 0:
                score = 0
                avg_divisor = 1
            avg_score = score / avg_divisor
            out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs]]))
            res.append((out, label))
    all_res.append(res)


In [16]:
# Evaluate combined result
ind = 0
length = 0
for res in all_res:
    length += len(res)
mini = 10000000000000000000 # arbitrarily big
for t in range (101):
    thres = t / 100
    mis = 0
    for res in all_res:
        for tup in res:
            if tup[0][0] > thres and tup[1] == 0:
                mis += 1
            elif tup[0][0] < thres and tup[1] == 1:
                mis += 1
    if mis < mini:
        ind = t
        mini = mis
print("Ideal threshold:", ind / 100)
print("Number misclassified:", mini)
print("Misclassification rate:", mini / length)

Ideal threshold: 0.3
Number misclassified: 331
Misclassification rate: 0.04740761959323976


# All Datasets

In [17]:
# Dataset 2 found at: https://www.kaggle.com/datasets/rrmartin/twitter-mental-disorder-tweets-and-musics

# Control data from dataset 2
control_filepath = "C:/Users/28chr/Downloads/anon_control_tweets.tar.xz/anon_control_tweets/anon_control_tweets.csv"
data2 = pd.read_csv(control_filepath)

# Disorder data from dataset 2
disorder_filepath = "C:/Users/28chr/Downloads/anon_disorder_tweets.tar.xz/anon_disorder_tweets/anon_disorder_tweets.csv"
data3 = pd.read_csv(disorder_filepath)

In [18]:
# Trim the data to save time while testing (optional)
# Anxiety data in the disorder dataset is grouped at the top of the file
max_len = 100000
data2["text"] = data2["text"][:max_len] 
data2["disorder"] = data2["disorder"][:max_len]
data3["text"] = data3["text"][:max_len]
data3["disorder"] = data3["disorder"][:max_len]

In [20]:
# Start a new word list with the words from dataset 1
word_list = {}
for i in range(len(data1["text"])):
    # i % 5 == 4 is the test set
    if i % 5 != 4:
        post = str(data1["text"][i])
        anxious = data1["label"][i] == 1
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += (letter)
            else:
                no_punct += (" ")
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                if anxious:
                    word_list[word] = (word_list[word][0], word_list[word][1] + 1)
                else:
                    word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                if anxious:
                    word_list[word] = (0, 1)
                else:
                    word_list[word] = (1, 0)

In [21]:
# Add dataset 2 to the word list

# control data
for i in range(len(data2["text"])):
    if i % 5 != 4 and data2["disorder"][i] == "control": # dataset should be control only
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase or letter in string.ascii_uppercase:
                no_punct += letter
            else:
                no_punct += " "
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                word_list[word] = (word_list[word][0] + 1, word_list[word][1])
            else:
                word_list[word] = (1, 0)
                
# disorder data
for i in range(len(data3["text"])):
    if i % 5 != 4 and data3["disorder"][i] == "anxiety": # dataset should be anxiety only
        post = str(data3["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase or letter in string.ascii_uppercase:
                no_punct += letter
            else:
                no_punct += " "
        for word in no_punct.split(" "):
            word = word.lower()
            if word in word_list:
                word_list[word] = (word_list[word][0], word_list[word][1] + 1)
            else:
                word_list[word] = (0, 1)

In [22]:
# Build anxiety scores for the new word list
anxiety_scores = {}
for word in word_list:
    num_control = word_list[word][0]
    num_anxious = word_list[word][1]
    num_total = num_control + num_anxious
    thresh = 1
    if num_total > thresh:
        score = num_anxious / num_total
        anxiety_scores[word] = score

In [23]:
# Experimentation revealed a need for a neutral words list
# for the larger dataset. Neutral words do not strongly indicate
# whether or not a person is anxious.

# Calculate average and standard deviation for word scores
scores_list = []
for word in anxiety_scores:
    scores_list.append(anxiety_scores[word])
avg = np.mean(scores_list)
std = np.std(scores_list)
lower = avg - std
upper = avg + std

# Make a list of words within one standard deviation of the mean
neutral_words = []
for word in anxiety_scores:
    score = anxiety_scores[word]
    if score > lower and score < upper:
        neutral_words.append(word)
# Remove these neutral words from the anxiety scores list
for word in neutral_words:
    anxiety_scores.pop(word)

In [25]:
# Calculate training scores and labels for Dataset 1
training_scores = []
training_labels = []
for i in range(len(data1["text"])):
    if i % 5 != 4:
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(data1["label"][i])
        post = str(data1["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])

In [26]:
# Add dataset 2 to the training set.

# control data
for i in range(len(data2["text"])):
    if i % 5 != 4 and (data2["disorder"][i] == "control"):
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(0)
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])
        
# disorder data
for i in range(len(data3["text"])):
    if i % 5 != 4 and (data3["disorder"][i] == "anxiety"):
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        training_labels.append(1)
        post = str(data3["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        training_scores.append([avg_score, caps, punct, len(post), one_offs, neutral])

In [27]:
# Clean the data
for i in range(len(training_scores)):
    if math.isnan(training_scores[i][0]):
        training_scores[i][0] = 0
    if math.isnan(training_labels[i]):
        training_labels[i] = 0

In [28]:
# Train a new linear regression model
training_scores = np.array(training_scores)
training_labels = np.array(training_labels)
reg = LinearRegression().fit(training_scores, training_labels)

In [30]:
# Get results from Dataset 1 test set
res = []
for i in range(len(data1["text"])):
    if i % 5 == 4:
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        label = data1["label"][i]
        post = str(data1["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

In [31]:
# Get results from Dataset 2 test set

# control data
for i in range(len(data2["text"])):
    if i % 5 == 4 and data2["disorder"][i] == "control":
        caps = 0
        punct = 0
        one_offs = 0
        label = 0
        neutral = 0
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                if word in neutral_words:
                    neutral += 1
                else:
                    one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

# disorder data
for i in range(len(data3["text"])):
    if i % 5 == 4 and data3["disorder"][i] == "anxiety":
        caps = 0
        punct = 0
        one_offs = 0
        neutral = 0
        label = 1
        post = str(data2["text"][i])
        no_punct = ""
        for letter in post:
            if letter in string.ascii_lowercase:
                no_punct += (letter)
            elif letter in string.ascii_uppercase:
                no_punct += letter
                caps += 1
            else:
                no_punct += " "
                punct += 1
        score = 0.0
        words = no_punct.split(" ")
        avg_divisor = len(words)
        for word in words:
            word = word.lower()
            if word in anxiety_scores:
                score += anxiety_scores[word]
            else:
                avg_divisor -= 1
                one_offs += 1
        if avg_divisor == 0:
            score = 0
            avg_divisor = 1
        avg_score = score / avg_divisor
        out = reg.predict(np.array([[avg_score, caps, punct, len(post), one_offs, neutral]]))
        res.append((out, label))

In [37]:
# Test the regression model.
# Experimentation revealed the need for a third classification, Uncertain,
# which must be factored into error calculation.

# Number of uncertain results that equal one misclassification for error calculation
# Depends on real-world utility values. Almost always converges to very high uncertainty
# and low misclassification, even for low values like 1.1.
# Choose whichever values provide the best balance between uncertainty and misclassification.
unc_divisor = 5

# Show results for different uncertainty values, and the ideal threshold for each.
for u in range(31):
    # Region around the threshold where results are labeled as uncertain
    uncertainty = u / 100
    # Keep track of information for the best run of each uncertainty value
    ind = 0
    mini = 10000000000000000000 # arbitrarily big
    mini_mis = -1
    fp = 0
    fn = 0
    unc = 0
    for t in range (-100, 100):
        # Ideal threshold tends to be 0.3, same as it was with just Dataset 1.
        thres = t / 100
        mis = 0
        false_positive = 0
        false_negative = 0
        uncertain = 0
        for tup in res:
            if tup[0][0] > thres and tup[1] == 0:
                if tup[0][0] > thres + uncertainty:
                    mis += 1
                    false_positive += 1
                else:
                    uncertain += 1
            elif tup[0][0] < thres and tup[1] == 1:
                if tup[0][0] < thres - uncertainty:
                    mis += 1
                    false_negative += 1
                else:
                    uncertain += 1
        this_error = mis + (uncertain / unc_divisor)
        if this_error < mini:
            mini = this_error
            ind = t
            mini_mis = mis
            fn = false_negative
            fp = false_positive
            unc = uncertain
    print("Uncertainty:", uncertainty)
    print("Ideal threshold:", ind / 100)
    print("Number misclassified:", mini_mis)
    print("Misclassification rate:", mini_mis / len(res))
    print("False positives:", fp)
    print("False negatives:", fn)
    print("Uncertain:", unc)
    print("Uncertainty rate:", unc / len(res))
    print("Error:", mini)
    print()

Uncertainty: 0.0
Ideal threshold: 30
Number misclassified: 14873
Misclassification rate: 0.3592859213450575
False positives: 5953
False negatives: 8920
Uncertain: 0
Uncertainty rate: 0.0
Error: 14873.0

Uncertainty: 0.01
Ideal threshold: 30
Number misclassified: 11332
Misclassification rate: 0.273746255676877
False positives: 3868
False negatives: 7464
Uncertain: 3541
Uncertainty rate: 0.0855396656681805
Error: 11686.1

Uncertainty: 0.02
Ideal threshold: 30
Number misclassified: 9437
Misclassification rate: 0.22796888588269398
False positives: 2852
False negatives: 6585
Uncertain: 5436
Uncertainty rate: 0.1313170354623635
Error: 9980.6

Uncertainty: 0.03
Ideal threshold: 29
Number misclassified: 7072
Misclassification rate: 0.1708377621026186
False positives: 2852
False negatives: 4220
Uncertain: 9682
Uncertainty rate: 0.23388733210938256
Error: 8040.2

Uncertainty: 0.04
Ideal threshold: 29
Number misclassified: 5508
Misclassification rate: 0.13305633394530872
False positives: 2355
Fal